### Data merge - Only pollutants O3, SO2, CO, NO2 and PM25 with target counties

In [ ]:
import pandas as pd
import glob, os

# 1) Config
data_dir = 'data_2025'
target_states = ['06','36','48','17']
target_counties = {
    '06': ['037','075','073'],    # CA metros
    '36': ['061','001'],          # NY metros
    '48': ['201','113'],          # TX metros
    '17': ['031']                 # IL metros
}
pollutants = {
    '44201': 'O3',
    '42401': 'SO2',
    '42101': 'CO',
    '42602': 'NO2',
    '88101': 'PM25'
}

# Pre-compute valid (state,county) strings
valid_pairs = {
    f"{st}_{cty}"
    for st, counties in target_counties.items()
    for cty in counties
}

# 2) Chunked loader + filter
def load_and_filter(path_zip, pollutant_name):
    usecols = ['State Code','County Code','Site Num','Date Local','Arithmetic Mean']
    reader = pd.read_csv(
        path_zip,
        usecols=usecols,
        parse_dates=['Date Local'],
        dtype={'State Code': str, 'County Code': str},
        compression='zip',
        chunksize=500_000,
        low_memory=False
    )
    
    chunks = []
    for chunk in reader:
        # zero-pad codes to match AQS formatting
        chunk['State Code']  = chunk['State Code'].str.zfill(2)
        chunk['County Code'] = chunk['County Code'].str.zfill(3)
        
        # keep only the target states
        chunk = chunk[chunk['State Code'].isin(target_states)]
        
        # filter by valid (state,county)
        chunk['pair'] = chunk['State Code'] + '_' + chunk['County Code']
        chunk = chunk[chunk['pair'].isin(valid_pairs)]
        chunk.drop(columns='pair', inplace=True)
        
        # rename pollutant column
        chunk = chunk.rename(columns={'Arithmetic Mean': pollutant_name})
        chunks.append(chunk)
    
    return pd.concat(chunks, ignore_index=True)


# 3) Merge all pollutants iteratively
merged = None
for code, name in pollutants.items():
    zip_path = os.path.join(data_dir, f'daily_{code}_2025.zip')
    df = load_and_filter(zip_path, name)
    
    if merged is None:
        merged = df
    else:
        merged = merged.merge(
            df,
            on=['State Code','County Code','Site Num','Date Local'],
            how='outer'
        )
    
    del df

# 4) Post-processing
merged = merged.sort_values(
    ['State Code','County Code','Site Num','Date Local']
)
merged.fillna(method='ffill', inplace=True)

# 5) Save
merged.to_csv('daily_2025_multi_pollutant.csv', index=False)
print("Merged data written to daily_2025_multi_pollutant.csv")


Merged data written to daily_2025_multi_pollutant.csv


/var/folders/rx/gyp8p3vs42zfzk_hwhrnpcmw0000gn/T/ipykernel_5733/1729318645.py:84: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged.fillna(method='ffill', inplace=True)


### Data merge - Pollutants O3, SO2, CO, NO2, PM25, meteorlogical data and air quality index with target counties

In [2]:
import pandas as pd
import os, glob

# 1. Configuration
data_dir = '/Users/amalshar/Desktop/USD/Fall 2025/Data Analysis/data preparation/data_2025'  
output_csv = 'daily_2025_full_merge.csv'

# Target geography
target_states = ['06','36','48','17']
target_counties = {
    '06': ['037','075','073'],   # CA metros
    '36': ['061','001'],         # NY metros
    '48': ['201','113'],         # TX metros
    '17': ['031']                # IL metros
}
valid_pairs = {
    f"{st}_{cty}"
    for st, counties in target_counties.items()
    for cty in counties
}

# Mapping zip → parameter codes → output column names
file_params = {
    'daily_42101_2025.zip': {'42101':'CO'},
    'daily_42401_2025.zip': {'42401':'SO2'},
    'daily_42602_2025.zip': {'42602':'NO2'},
    'daily_44201_2025.zip': {'44201':'O3'},
    'daily_88101_2025.zip': {'88101':'PM25'},
    'daily_PRESS_2025.zip':   {'64101':'BarometricPressure'},
    'daily_TEMP_2025.zip':    {'62101':'Temperature'},
    'daily_WIND_2025.zip':    {'62201':'WindResultant'},
    'daily_RH_DP_2025.zip':   {'62202':'RelativeHumidity','62205':'DewPoint'}
}

# 2. Single‐code loader
def load_single(zip_path, code, name):
    usecols = ['State Code','County Code','Site Num','Date Local','Arithmetic Mean']
    reader = pd.read_csv(
        zip_path, usecols=usecols,
        dtype={'State Code': str, 'County Code': str},
        parse_dates=['Date Local'],
        compression='zip', chunksize=300_000, low_memory=False
    )
    parts = []
    for ch in reader:
        ch['State Code']  = ch['State Code'].str.zfill(2)
        ch['County Code'] = ch['County Code'].str.zfill(3)
        ch = ch[ch['State Code'].isin(target_states)]
        ch['pair'] = ch['State Code'] + '_' + ch['County Code']
        ch = ch[ch['pair'].isin(valid_pairs)].drop(columns='pair')
        ch = ch.rename(columns={'Arithmetic Mean': name})
        parts.append(ch)
    return pd.concat(parts, ignore_index=True)

# 3. Multi‐code loader (for RH & DewPoint)
def load_multi(zip_path, code_map):
    usecols = ['State Code','County Code','Site Num','Date Local',
               'Parameter Code','Arithmetic Mean']
    reader = pd.read_csv(
        zip_path, usecols=usecols,
        dtype={'State Code': str, 'County Code': str, 'Parameter Code': str},
        parse_dates=['Date Local'],
        compression='zip', chunksize=300_000, low_memory=False
    )
    parts = []
    keep_codes = set(code_map.keys())
    for ch in reader:
        ch['State Code']  = ch['State Code'].str.zfill(2)
        ch['County Code'] = ch['County Code'].str.zfill(3)
        ch = ch[ch['State Code'].isin(target_states)]
        ch['pair'] = ch['State Code'] + '_' + ch['County Code']
        ch = ch[ch['pair'].isin(valid_pairs)]
        ch = ch[ch['Parameter Code'].isin(keep_codes)]
        # pivot Parameter Code → column
        piv = ch.pivot_table(
            index=['State Code','County Code','Site Num','Date Local'],
            columns='Parameter Code',
            values='Arithmetic Mean',
            aggfunc='first'
        )
        piv = piv.rename(columns=code_map).reset_index()
        parts.append(piv)
    return pd.concat(parts, ignore_index=True)

# 4. Iterate and merge all
merged = None
for fname, cmap in file_params.items():
    path = os.path.join(data_dir, fname)
    print(f"→ Loading {fname}")
    if len(cmap) == 1:
        code, col = next(iter(cmap.items()))
        df = load_single(path, code, col)
    else:
        df = load_multi(path, cmap)
    if merged is None:
        merged = df
    else:
        merged = merged.merge(
            df,
            on=['State Code','County Code','Site Num','Date Local'],
            how='outer'
        )
    del df

# 5. Merge daily AQI by county (no Site Num)
aqi_zip = os.path.join(data_dir, 'daily_aqi_by_county_2025.zip')
print("→ Loading daily AQI by county")
aqi = pd.read_csv(
    aqi_zip,
    dtype={'State Code': str, 'County Code': str},
    parse_dates=['Date'],
    compression='zip',
    low_memory=False
)
aqi['State Code']  = aqi['State Code'].str.zfill(2)
aqi['County Code'] = aqi['County Code'].str.zfill(3)
aqi = aqi[aqi['State Code'].isin(target_states)]
aqi = aqi.rename(columns={'Date':'Date Local','AQI':'DailyAQI'})
merged = merged.merge(
    aqi[['State Code','County Code','Date Local','DailyAQI']],
    on=['State Code','County Code','Date Local'],
    how='left'
)

# 6. Final cleanup & save
merged = merged.sort_values(['State Code','County Code','Site Num','Date Local'])
merged.fillna(method='ffill', inplace=True)
merged.to_csv(output_csv, index=False)
print(f"Finished! Output saved to {output_csv}")


→ Loading daily_42101_2025.zip
→ Loading daily_42401_2025.zip
→ Loading daily_42602_2025.zip
→ Loading daily_44201_2025.zip
→ Loading daily_88101_2025.zip
→ Loading daily_PRESS_2025.zip
→ Loading daily_TEMP_2025.zip
→ Loading daily_WIND_2025.zip
→ Loading daily_RH_DP_2025.zip
→ Loading daily AQI by county
Finished! Output saved to daily_2025_full_merge.csv


/var/folders/rx/gyp8p3vs42zfzk_hwhrnpcmw0000gn/T/ipykernel_5733/649074219.py:127: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged.fillna(method='ffill', inplace=True)


### Data merge - Pollutants O3, SO2, CO, NO2, PM25, meteorlogical data and air quality index with all counties

In [ ]:
import pandas as pd
import os

# 1. Configuration
data_dir    = '/Users/amalshar/Desktop/USD/Fall 2025/Data Analysis/data preparation/data_2025'  
output_csv  = 'daily_2025_full_merge_all_counties.csv'

# keep only these states, but include all counties
target_states = ['06','36','48','17']

# map each ZIP filename to its parameter codes → output column names
file_params = {
    'daily_42101_2025.zip': {'42101':'CO'},
    'daily_42401_2025.zip': {'42401':'SO2'},
    'daily_42602_2025.zip': {'42602':'NO2'},
    'daily_44201_2025.zip': {'44201':'O3'},
    'daily_88101_2025.zip': {'88101':'PM25'},
    'daily_PRESS_2025.zip':  {'64101':'BarometricPressure'},
    'daily_TEMP_2025.zip':   {'62101':'Temperature'},
    'daily_WIND_2025.zip':   {'62201':'WindResultant'},
    'daily_RH_DP_2025.zip':  {'62202':'RelativeHumidity','62205':'DewPoint'}
}

# 2. Loader for single-code files
def load_single(zip_path, code, name):
    usecols = ['State Code','County Code','Site Num','Date Local','Arithmetic Mean']
    reader = pd.read_csv(
        zip_path, usecols=usecols,
        dtype={'State Code': str, 'County Code': str},
        parse_dates=['Date Local'],
        compression='zip', chunksize=300_000, low_memory=False
    )
    parts = []
    for chunk in reader:
        # zero-pad to match AQS formatting
        chunk['State Code']  = chunk['State Code'].str.zfill(2)
        chunk['County Code'] = chunk['County Code'].str.zfill(3)

        # keep only our 4 states
        chunk = chunk[chunk['State Code'].isin(target_states)]

        # rename and collect
        chunk = chunk.rename(columns={'Arithmetic Mean': name})
        parts.append(chunk)

    return pd.concat(parts, ignore_index=True)

# 3. Loader for multi-code files (RH & DewPoint)
def load_multi(zip_path, code_map):
    usecols = ['State Code','County Code','Site Num','Date Local','Parameter Code','Arithmetic Mean']
    reader = pd.read_csv(
        zip_path, usecols=usecols,
        dtype={'State Code': str, 'County Code': str, 'Parameter Code': str},
        parse_dates=['Date Local'],
        compression='zip', chunksize=300_000, low_memory=False
    )
    parts = []
    keep_codes = set(code_map.keys())
    for chunk in reader:
        chunk['State Code']  = chunk['State Code'].str.zfill(2)
        chunk['County Code'] = chunk['County Code'].str.zfill(3)
        chunk = chunk[chunk['State Code'].isin(target_states)]
        chunk = chunk[chunk['Parameter Code'].isin(keep_codes)]

        # pivot codes into columns
        piv = chunk.pivot_table(
            index=['State Code','County Code','Site Num','Date Local'],
            columns='Parameter Code',
            values='Arithmetic Mean',
            aggfunc='first'
        ).rename(columns=code_map).reset_index()

        parts.append(piv)

    return pd.concat(parts, ignore_index=True)

# 4. Iteratively load & merge all parameter files
merged = None
for fname, cmap in file_params.items():
    path = os.path.join(data_dir, fname)
    print(f"Loading {fname} …")

    if len(cmap) == 1:
        code, col = next(iter(cmap.items()))
        df = load_single(path, code, col)
    else:
        df = load_multi(path, cmap)

    if merged is None:
        merged = df
    else:
        merged = merged.merge(
            df,
            on=['State Code','County Code','Site Num','Date Local'],
            how='outer'
        )
    del df  

# 5. Merge county-level Daily AQI (no Site Num)
aqi_zip = os.path.join(data_dir, 'daily_aqi_by_county_2025.zip')
print("Loading daily AQI by county …")
aqi = pd.read_csv(
    aqi_zip,
    dtype={'State Code': str, 'County Code': str},
    parse_dates=['Date'],
    compression='zip', low_memory=False
)
aqi['State Code']  = aqi['State Code'].str.zfill(2)
aqi['County Code'] = aqi['County Code'].str.zfill(3)
aqi = aqi[aqi['State Code'].isin(target_states)]
aqi = aqi.rename(columns={'Date':'Date Local','AQI':'DailyAQI'})

merged = merged.merge(
    aqi[['State Code','County Code','Date Local','DailyAQI']],
    on=['State Code','County Code','Date Local'],
    how='left'
)

# 6. Final cleanup & save
merged = merged.sort_values(['State Code','County Code','Site Num','Date Local'])
merged.fillna(method='ffill', inplace=True)
merged.to_csv(output_csv, index=False)

print(f"All counties in {target_states} merged into {output_csv}")


Loading daily_42101_2025.zip …
Loading daily_42401_2025.zip …
Loading daily_42602_2025.zip …
Loading daily_44201_2025.zip …
Loading daily_88101_2025.zip …
Loading daily_PRESS_2025.zip …
Loading daily_TEMP_2025.zip …
Loading daily_WIND_2025.zip …
Loading daily_RH_DP_2025.zip …
Loading daily AQI by county …


/var/folders/rx/gyp8p3vs42zfzk_hwhrnpcmw0000gn/T/ipykernel_5733/3400328356.py:121: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged.fillna(method='ffill', inplace=True)


All counties in ['06', '36', '48', '17'] merged into daily_2025_full_merge_all_counties.csv
